<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">

# Reinforcement Learning

## OpenAI Gym

Dr Yves J Hilpisch | The AI Machine

http://aimachine.io | http://twitter.com/dyjh

## CartPole

In [ ]:
!git clone https://github.com/tpq-classes/rl_for_finance.git
import sys
sys.path.append('rl_for_finance')


In [ ]:
!pip install pyvirtualdisplay


In [ ]:
import gymnasium as gym
import numpy as np
import pandas as pd
from pylab import plt
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()
from IPython import display
plt.ion()
plt.style.use('seaborn-v0_8')
%matplotlib inline
np.random.seed(100)

In [ ]:
env = gym.make('CartPole-v1', render_mode='rgb_array')

In [ ]:
env.action_space.seed(100)

## Action Space

In [ ]:
env.action_space  # type of action space

In [ ]:
env.action_space.n  # number of possible actions

In [ ]:
env.action_space.sample()  # sample action | move left = 0 | move right = 1

In [ ]:
env.action_space.sample()  # sample action | move left = 0 | move right = 1

In [ ]:
[env.action_space.sample() for _ in range(25)]

## Observation Space

In [ ]:
np.set_printoptions(precision=4, suppress=True)

In [ ]:
env.observation_space  # type of observation space

In [ ]:
env.observation_space.high.astype(np.float16) # upper bounds for observations

In [ ]:
env.observation_space.low.astype(np.float16)  # lower bounds for observations

In [ ]:
o = env.reset()
o  # [cart position, cart velocity, pole angle, pole angular velocity]

## Taking Action

The following visualizes the effect of a number of random actions taken. See https://gist.github.com/thomelane/79e97630ba46c45985a946cae4805885

In [ ]:
a = env.action_space.sample()  # random action
a

In [ ]:
r = env.step(a)  # taking action, capturing new observations
r  # (observation, reward, done, info)

In [ ]:
env.reset()
img = plt.imshow(env.render()) # initialize bitmap embedding
for e in range(201):
    img.set_data(env.render()) # updating the data
    display.display(plt.gcf())
    display.clear_output(wait=True)
    a = env.action_space.sample()  # random action choice
    obs, rew, done, _, _ = env.step(a)  # taking action
    if done and (e + 1) < 200:
        print('*** FAILED ***')
        break

## Simple Regression Approach

### Dimensionality Reduction

By using four weights and taking the dot product between the weights and the four observation values, the observation (state) space can be reduced from 4 dimensions to just 1. See http://kvfrans.com/simple-algoritms-for-solving-cartpole/.

In [ ]:
weights = np.random.random(4) * 2 - 1  # 4 random weights ...

In [ ]:
weights  # ... between -1 and 1

In [ ]:
o, _ = env.reset()

In [ ]:
o  # reduction of dimensionality from 4 ...

In [ ]:
s = np.dot(weights, o)  # ... to 1
s

### Action Rule

The agent behaves according to the following action rule:

In [ ]:
if s < 0:  # if single state value is negative ...
    a = 0  # ... move left
else:  # otherwise ...
    a = 1  # ... move right

In [ ]:
a

### Learning Objective

Learn those `weights` that allow the agent to survive 200 steps based on the above action rule.

### Total Reward per Episode

Function that returns the total reward `trew` given certain `weights` and the action rule.

In [ ]:
def run_episode(env, weights):
    o, _ = env.reset()
    trew = 0
    for _ in range(200):
        s = np.dot(weights, o)
        a = 0 if s < 0 else 1
        o, rew, done, trunc, info = env.step(a)
        trew += rew
        if done:
            break
    return trew

In [ ]:
run_episode(env, weights)  # 200 means success

### Simple Learning

The following code runs a maximum number of episodes `num_episodes` and stops when a certain `weights` combination makes the agent successful (200 survived actions).

In [ ]:
num_episodes = 5000

In [ ]:
%%time
bestweights = None
besttrew = 0
for _ in range(1, num_episodes + 1):
    print('episode = {}'.format(_), end='\r')
    weights = np.random.rand(4) * 2 - 1
    trew = run_episode(env, weights)
    if trew > besttrew:
        besttrew = trew
        bestweights = weights
        if trew == 200:  # success?
            break

In [ ]:
weights  # learned ('optimal') weights

### Testing the Results

In [ ]:
# some episodes with the learned weights
res = []
for _ in range(15):
    trew = run_episode(env, weights)
    res.append(int(trew))
res

In [ ]:
# a single episode visualized (inline)
o, _ = env.reset()
img = plt.imshow(env.render()) # initialize bitmap embedding
for _ in range(200):
    img.set_data(env.render()) # updating the data
    display.display(plt.gcf())
    display.clear_output(wait=True)
    s = np.dot(weights, o)
    a = 0 if s < 0 else 1
    o, rew, done, itrunc, info = env.step(a)  # taking action
    if done and (_ + 1) < 200:
        print('*** FAILED at STEP {} ***'.format(_ + 1))
        break
if not done:
    print('*** SUCCESS ***')

In [ ]:
# a single episode visualized (pop-up)
o, _ = env.reset()
for _ in range(1, 200 + 1):
    env.render()
    s = np.dot(weights, o)
    a = 0 if s < 0 else 1
    o, rew, done, trunc,  info = env.step(a)  # taking action
    print('{:3d} | state={:6.3f} | action={}'.format(_, s, a), end='\r')
    if done:
        if _ < 200:
            print('\n*** FAILED at STEP {} ***'.format(_ + 1))
        break

<img src='http://hilpisch.com/taim_logo.png' width="350px" align="right">

<br><br><br><a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:ai@tpq.io">ai@tpq.io</a>